In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time, datetime
import calendar
import warnings
warnings.filterwarnings('ignore')

#### Inventory Optimisation and Sustainability Analysis
#### Objectives:
   - Minimize waste by optimizing inventory levels
   - Identify the most efficient purchasing strategies based on sales, procurement and inventory data.
   - Evaluate product sales performance to formulate a sustainable inventory management approach

       1. Data Preprocessing
       2. Inventory Analysis
       3. Sales Analysis
       4. Purchasing Analysis
       5. Optimal Stock Level Calculation

In [9]:
#load datasets
purchase_prices_data = pd.read_csv('2017PurchasePricesDec.csv')
start_invoice_data = pd.read_csv('BegInvFINAL12312016.csv')
end_invoice_data = pd.read_csv('EndInvFINAL12312016.csv')
invoice_purchase_data = pd.read_csv('InvoicePurchases12312016.csv')
purchases_data = pd.read_csv('PurchasesFINAL12312016.csv')
sales_data = pd.read_csv('SalesFINAL12312016.csv')


In [3]:
purchase_prices_data.head()

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


In [4]:
start_invoice_data.head()

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2016-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2016-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2016-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2016-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2016-01-01


In [5]:
end_invoice_data.head()

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2016-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2016-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2016-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2016-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2016-12-31


In [6]:
invoice_purchase_data.head()

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2016-01-04,8124,2015-12-21,2016-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2016-01-07,8137,2015-12-22,2016-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2016-01-09,8169,2015-12-24,2016-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2016-01-12,8106,2015-12-20,2016-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2016-01-07,8170,2015-12-24,2016-02-12,1935,15527.25,429.20,None


In [7]:
purchases_data.head()

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2015-12-21,2016-01-02,2016-01-04,2016-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-02,2016-01-07,2016-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2015-12-22,2016-01-01,2016-01-07,2016-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2015-12-24,2016-01-02,2016-01-09,2016-02-16,21.32,5,106.60,1


In [11]:
sales_data.head()

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/1/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,1/2/2016,750,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/3/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,1/8/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,1/9/2016,375,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [ ]:
# #load datasets
# purchase_prices_data = pd.read_csv('2017PurchasePricesDec.csv')
# start_invoice_data = pd.read_csv('BegInvFINAL12312016.csv')
# end_invoice_data = pd.read_csv('EndInvFINAL12312016.csv')
# invoice_purchase_data = pd.read_csv('InvoicePurchases12312016.csv')
# purchases_data = pd.read_csv('PurchasesFINAL12312016.csv')
# sales_data = pd.read_csv('SalesFINAL12312016.csv')

In [38]:
# check for missing values in each data
all_data = [
            purchase_prices_data,
            start_invoice_data,
            end_invoice_data,
            invoice_purchase_data,
            sales_data
]

all_data_names = ["purchase_prices_data", 
                 "start_invoice_data",
                 "end_invoice_data",
                 "invoice_purchase_data",
                 "sales_data"]

for name, data in zip(all_data_names, all_data):
    missing_values = data.isna().sum()
    non_zero_missing_values = missing_values[missing_values > 0]
    print(f"\nMissing values in {name}:")
    print(non_zero_missing_values)
    
#     if not non_zero_missing_values.empty:
#         print(f"\nMissing values in {name}:")
#         print(non_zero_missing_values)


Missing values in purchase_prices_data:
Description    1
Size           1
Volume         1
dtype: int64

Missing values in start_invoice_data:
Series([], dtype: int64)

Missing values in end_invoice_data:
City    1284
dtype: int64

Missing values in invoice_purchase_data:
Series([], dtype: int64)

Missing values in sales_data:
Series([], dtype: int64)


#Exploring Dataset One after the other
### Purchase Prices

Check for Null Values:


In [39]:
purchase_prices_data.isna().sum()

Brand             0
Description       1
Price             0
Size              1
Volume            1
Classification    0
PurchasePrice     0
VendorNumber      0
VendorName        0
dtype: int64

In [ ]:
#drop na values

In [35]:
 #load datasets
# purchase_prices_data.isna().sum()
# start_invoice_data.isna().sum() no
# end_invoice_data.isna().sum()
# invoice_purchase_data.isna().sum() no
# purchases_data.isna().sum() 3
sales_data.isna().sum()

InventoryId       0
Store             0
Brand             0
Description       0
Size              0
SalesQuantity     0
SalesDollars      0
SalesPrice        0
SalesDate         0
Volume            0
Classification    0
ExciseTax         0
VendorNo          0
VendorName        0
dtype: int64

In [1]:
# git remote add origin https://github.com/BossLadyZ/Inventory-Data-Analysis.git
# git branch -M main
# git push -u origin main

##  Inventory-Data-Analysis

#### Inventory Optimisation and Sustainability Analysis
#### Objectives:
   - Minimize waste by optimizing inventory levels
   - Identify the most efficient purchasing strategies based on sales, procurement and inventory data.
   - Evaluate product sales performance to formulate a sustainable inventory management approach

       1. Data Preprocessing
       2. Inventory Analysis
       3. Sales Analysis
       4. Purchasing Analysis
       5. Optimal Stock Level Calculation
       6. 
#### Tools:
   - Python, Numpy, Pandas, Matplotlib, Seaborn
    


### Uber Data Analysis

This document contains the exploratory data analysis of Uber Data collected in 2016. It contains 1155 data of a single Uber user in 2016. The features of the dataset are trip date, destination, source, distance travelled and purpose of the trip.

Uber is an online ride-hailing service with over 118 million users, 5 million drivers, and 6.3 billion trips with 17.4 million trips completed daily. 

This analysis was conducted to answer questions like:
  - peak ride periods
  - Busy areas/routes
  - Most Ride purposes, etc
Insights generated at the end of the analysis could be used to make better marketing suggestions and campaigns to increase the customer base and get more drivers to enrol as drivers on the platform.

Tools used:
  - Python
  - Numpy
  - Pandas
  - Matplotlib
